In [72]:
import pandas as pd 
import pandas_ta as ta 
import yfinance as yf 
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

In [51]:
def chart(df):

    fig = go.Figure(
        data=[go.Candlestick(
            x=df['date'],
            open=df['open'],
            high=df['high'],
            low=df['low'],
            close=df['close']
        )]
    )

    fig.update_layout(
        xaxis_rangeslider_visible = False
    )
    
    pio.show(fig, renderer='browser') 

In [115]:
def cleaning(df):
    df = df.copy()  # Make a copy to avoid modifying the original DataFrame
    df.columns = [name[0].lower() for name in df.columns]  # Convert column names to lowercase
    df.index = df.index.rename('date')  # Rename index to 'date'
    df = df.reset_index(drop=False)
    df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d %H:%M:%S')  # Format index as datetime string
    return df

In [121]:
def d1_fibonacci(df):
    # Convert 'date' column to datetime and extract the date part
    df['date'] = pd.to_datetime(df['date'])

    # Group by date and calculate OHLC and volume
    daily_df = df.groupby('date').agg({
        'open': 'first',
        'high': 'max',
        'low': 'min',
        'close': 'last',
        'volume': 'sum'
    })

    # Calculate Fibonacci level (61.8% retracement)
    daily_df['fib_61_8'] = daily_df['high'] - (daily_df['high'] - daily_df['low']) * 0.618

    # Reset index and ensure the 'date' column is in datetime format
    daily_df = daily_df.reset_index()
    daily_df['date'] = pd.to_datetime(daily_df['date']).dt.strftime('%Y-%m-%d %H:%M:%S')

    # Merge the 'fib_61_8' column back into the original DataFrame
    # df = pd.merge(
    #     df,
    #     daily_df[['date', 'fib_61_8']],
    #     on='date',
    #     how='outer'
    # )

    return daily_df, df


In [122]:
gold_15 = cleaning(yf.download('GC=F',period='8d', interval='15m'))

[*********************100%***********************]  1 of 1 completed


In [123]:
gold_15.head(3).loc[:]

,date,close,high,low,open,volume
0,2025-01-20 00:00:00,2747.199951,2748.000000,2746.699951,2747.100098,183
1,2025-01-20 00:15:00,2748.199951,2748.600098,2747.100098,2747.100098,313
2,2025-01-20 00:30:00,2749.000000,2749.199951,2746.899902,2748.100098,582


In [124]:
d1_fibonacci(gold_15)[0]

,date,open,high,low,close,volume,fib_61_8
0,2025-01-20 00:00:00,2747.100098,2748.000000,2746.699951,2747.199951,183,2747.196570
1,2025-01-20 00:15:00,2747.100098,2748.600098,2747.100098,2748.199951,313,2747.673098
2,2025-01-20 00:30:00,2748.100098,2749.199951,2746.899902,2749.000000,582,2747.778521
3,2025-01-20 00:45:00,2748.899902,2749.300049,2746.800049,2749.100098,1296,2747.755049
4,2025-01-20 01:00:00,2749.000000,2750.500000,2748.300049,2749.699951,623,2749.140430
...,...,...,...,...,...,...,...
599,2025-01-28 14:15:00,2772.199951,2773.199951,2771.300049,2773.100098,811,2772.025812
600,2025-01-28 14:30:00,2773.000000,2774.000000,2772.199951,2772.500000,1910,2772.887570
601,2025-01-28 14:45:00,2772.600098,2773.500000,2772.199951,2773.199951,1353,2772.696570
602,2025-01-28 15:00:00,2773.199951,2773.600098,2771.899902,2773.000000,983,2772.549377


In [125]:
d1_fibonacci(gold_15)[1]

,date,close,high,low,open,volume
0,2025-01-20 00:00:00,2747.199951,2748.000000,2746.699951,2747.100098,183
1,2025-01-20 00:15:00,2748.199951,2748.600098,2747.100098,2747.100098,313
2,2025-01-20 00:30:00,2749.000000,2749.199951,2746.899902,2748.100098,582
3,2025-01-20 00:45:00,2749.100098,2749.300049,2746.800049,2748.899902,1296
4,2025-01-20 01:00:00,2749.699951,2750.500000,2748.300049,2749.000000,623
...,...,...,...,...,...,...
599,2025-01-28 14:15:00,2773.100098,2773.199951,2771.300049,2772.199951,811
600,2025-01-28 14:30:00,2772.500000,2774.000000,2772.199951,2773.000000,1910
601,2025-01-28 14:45:00,2773.199951,2773.500000,2772.199951,2772.600098,1353
602,2025-01-28 15:00:00,2773.000000,2773.600098,2771.899902,2773.199951,983


In [56]:
chart(gold_15)